In [ ]:
import pandas as pd
from RoutePlanner.CellGrid import CellGrid
from matplotlib import pyplot as plt
import xarray as xr
import netCDF4 as nc
import datetime
from RoutePlanner.TemporalCellGrid import TemporalCellGrid
import matplotlib.pyplot as plt
from RoutePlanner.CellBox import CellBox
import pandas as pd
from RoutePlanner.Optimisation import TravelTime
from RoutePlanner import Plot
from RoutePlanner.TemporalCellGrid import TemporalCellGrid
import pickle

In [ ]:
OptInfo = {}

# Mesh Information
OptInfo['Mesh']  = {}
OptInfo['Mesh']['Longitude Bounds (Min,Max,Width)']      = [-130+0.0001,30+0.0001,5]
OptInfo['Mesh']['Latitude Bounds (Min,Max,Width)']       = [-80,-40,2.5]

OptInfo['Mesh']['Date Range (Min,Max,dT)']               = ['2013-01-1','2013-01-30',1.0]
OptInfo['Mesh']['Homogenous Params (Threshold,Min,Max)'] = [0.25,0.05,0.95]#[0.25,0.05,0.95] [0.35,0.05,0.99]#[0.04,0.05,0.85] - Slow Vehicle, [0.12,0.05,0.85] - SDA
OptInfo['Mesh']['Current Data Path']                     = "../../Data/SOSE_surface_velocity_6yearMean_2005-2010.nc"
OptInfo['Mesh']['Ice Data Path']                         = "../../Data/bsose_i122_2013to2017_1day_SeaIceArea.nc"

# Route Information
OptInfo['Route'] = {} 
OptInfo['Route']['WayPoints']            = '../../resources/Modis_Waypoints.csv'
OptInfo['Route']['MaxIceExtent']         = 0.8
OptInfo['Route']['Zero Currents']        = True
OptInfo['Route']['VariableSpeed']        = True
OptInfo['Route']['Time Unit']            = 'days'

OptInfo['Route']['VehicleInfo']                = {}
OptInfo['Route']['VehicleInfo']['Speed']       = 26.5
OptInfo['Route']['VehicleInfo']['Unit']        = 'km/hr'
OptInfo['Route']['VehicleInfo']['Beam']        = 24.0
OptInfo['Route']['VehicleInfo']['HullType']    = 'slender'
OptInfo['Route']['VehicleInfo']['ForceLimit']  = 96634.5

In [ ]:
def loadIce(icePointsPath, startTime, endTime):
    icePoints = xr.open_dataset(icePointsPath)
    ipSlice = icePoints.sel(time=slice(startTime, endTime))
    ipSlice = ipSlice.to_dataframe()
    
    ipSlice = ipSlice.reset_index()
    ipSlice = ipSlice.rename(columns={'XC':'long', 'YC':'lat', 'Depth':'depth', 'SIarea':'iceArea'})
    ipSlice['long'] = ipSlice['long'].apply(lambda x: x if x <= 180 else x - 360)
  
    return ipSlice


# === Manually Adding Currents
sose = nc.Dataset("../../Data/SOSE_surface_velocity_6yearMean_2005-2010.nc")
currentPoints = pd.DataFrame({'long': sose['lon'][...].data.flatten(),
                                'lat': sose['lat'][...].data.flatten(),
                                'uC': sose['uC'][...].data.flatten(),
                                'vC': sose['vC'][...].data.flatten()})

currentPoints['time'] = ''
currentPoints['uC'] = currentPoints['uC'] * 3.6
currentPoints['vC'] = currentPoints['vC'] * 3.6
currentPoints['long'] = currentPoints['long'].apply(lambda x: x if x <= 180 else x - 360)


# === Adding Modis Information
modis_ips = pd.read_csv('../../Data/Modis_1km.csv')
modis_ips = modis_ips.rename(columns ={'0':'long', '1':'lat', '2':'time', '3':'iceArea', '4':'cloud'})
modis_ips['iceArea'] = modis_ips['iceArea'] * 0.1
modis_ips = modis_ips[modis_ips.cloud != 1]
#modis_ips = modis_ips.drop(columns = ['Unnamed: 0', 'cloud'])

# ==== Loading BSOSE
icePointsPath = "../../Data/bsose_i122_2013to2017_1day_SeaIceArea.nc"
startTime = '2017-02-19'
endTime = '2017-02-19'
bsose = loadIce(icePointsPath, startTime, endTime)
bsose = bsose.drop(columns = ['iter', 'rA', 'iceArea'])

In [ ]:
latMin = -80
latMax = -60
longMin = -65
longMax = -10

cellWidth = 5
cellHeight = 2.5

splittingPercentage = 0.35 # thershold
splitMinProp = 0.3 # lowerbound
splitMaxProp = 1.0 # upperbound


modis_grid = CellGrid(longMin, longMax, latMin, latMax, cellWidth, cellHeight,j_grid=False)
modis_grid.addIcePoints(modis_ips)
modis_grid.addCurrentPoints(currentPoints)
modis_grid.addDataPoints(bsose)
modis_grid.iterativeSplit(6, splittingPercentage, splitMinProp, splitMaxProp)


In [ ]:
map = Plot.BaseMap('RoutePlanner Modis Overlay')
def MapGeotiff(map,file,subregion=None,name='Modis',resamplingFactor=1):
    import folium
    import rasterio as rs
    from matplotlib import cm
    from IceMDP.IO import windowFunc
    import numpy as np
    '''
        Issue - Resampling only working on WGS84 projections
    '''
    dataset = rs.open(file)

    if type(subregion)!=type(None):
        window,bounds = windowFunc(subregion[0], subregion[1], dataset)
        bnds = [[bounds[1], bounds[0]], [bounds[3], bounds[2]]]
    else:
        window=None
        bnds = [[dataset.bounds[1], dataset.bounds[0]], [dataset.bounds[3], dataset.bounds[2]]]

    trying =True
    indx = 1
    while trying:
        try:

            if indx==1:
                opc = 0.8
                shw = True
            else:
                opc = 0.7
                shw = False

            img = folium.raster_layers.ImageOverlay(
                name="{} - Band {}".format(name,indx),
                image=dataset.read(indx,window=window)[::resamplingFactor,::resamplingFactor],
                bounds=bnds,
                opacity=opc,
                mercator_project=True,
                pixelated=False,
                show = shw
            )
            img.add_to(map)
        except:
            trying=False
        indx+=1
    return map

map = MapGeotiff(map,'../../Data/Weddle_4326.tiff',resamplingFactor=10)
map = Plot.MapMesh(modis_grid,map,threshold=0.8)
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.LayerControl(map,collapsed=True)
map


In [ ]:
map.save('RoutePlannerModis.html')

In [ ]:
OptInfo = {}

# Mesh Information
OptInfo['Mesh']  = {}
OptInfo['Mesh']['Longitude Bounds (Min,Max,Width)']      = [-130+0.0001,30+0.0001,5]
OptInfo['Mesh']['Latitude Bounds (Min,Max,Width)']       = [-80,-40,2.5]

OptInfo['Mesh']['Date Range (Min,Max,dT)']               = ['2013-01-1','2013-01-30',1.0]
OptInfo['Mesh']['Homogenous Params (Threshold,Min,Max)'] = [0.25,0.05,0.95]#[0.25,0.05,0.95] [0.35,0.05,0.99]#[0.04,0.05,0.85] - Slow Vehicle, [0.12,0.05,0.85] - SDA
OptInfo['Mesh']['Current Data Path']                     = "../../Data/SOSE_surface_velocity_6yearMean_2005-2010.nc"
OptInfo['Mesh']['Ice Data Path']                         = "../../Data/bsose_i122_2013to2017_1day_SeaIceArea.nc"

# Route Information
OptInfo['Route'] = {} 
OptInfo['Route']['WayPoints']            = '../../resources/Modis_Waypoints.csv'
OptInfo['Route']['MaxIceExtent']         = 0.8
OptInfo['Route']['Zero Currents']        = True
OptInfo['Route']['VariableSpeed']        = True
OptInfo['Route']['Time Unit']            = 'days'

OptInfo['Route']['VehicleInfo']                = {}
OptInfo['Route']['VehicleInfo']['Speed']       = 26.5
OptInfo['Route']['VehicleInfo']['Unit']        = 'km/hr'
OptInfo['Route']['VehicleInfo']['Beam']        = 24.0
OptInfo['Route']['VehicleInfo']['HullType']    = 'slender'
OptInfo['Route']['VehicleInfo']['ForceLimit']  = 96634.5

TT = TravelTime(modis_grid,OptInfo)
Paths = TT.Paths(source_waypoints=['Halley'],verbose=True,multiprocessing=False)

In [ ]:
 
map = Plot.BaseMap('Route Paths from Halley')
map = MapGeotiff(map,'../../Data/Weddle_4326.tiff',resamplingFactor=10)
map = Plot.MapMesh(modis_grid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = Plot.MapPaths(Paths,map,PathPoints=False)
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.LayerControl(map,collapsed=True)
map

In [ ]:
map.save('RoutePlannerModis.html')

In [ ]:
SmoothedPaths = TT.PathSmoothing(maxiter=300,return_paths=True)

In [ ]:
map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(modis_grid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = Plot.MapPaths(SmoothedPaths,map,PathPoints=False)
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.LayerControl(map,collapsed=True)
map

In [ ]:
OptInfo = {}

# Mesh Information
OptInfo['Mesh']  = {}
OptInfo['Mesh']['Longitude Bounds (Min,Max,Width)']      = [-130+0.0001,30+0.0001,5]
OptInfo['Mesh']['Latitude Bounds (Min,Max,Width)']       = [-80,-40,2.5]

OptInfo['Mesh']['Date Range (Min,Max,dT)']               = ['2013-01-1','2013-01-30',1.0]
OptInfo['Mesh']['Homogenous Params (Threshold,Min,Max)'] = [0.25,0.05,0.95]#[0.25,0.05,0.95] [0.35,0.05,0.99]#[0.04,0.05,0.85] - Slow Vehicle, [0.12,0.05,0.85] - SDA
OptInfo['Mesh']['Current Data Path']                     = "../../Data/SOSE_surface_velocity_6yearMean_2005-2010.nc"
OptInfo['Mesh']['Ice Data Path']                         = "../../Data/bsose_i122_2013to2017_1day_SeaIceArea.nc"

# Route Information
OptInfo['Route'] = {} 
OptInfo['Route']['WayPoints']            = '../../resources/Modis_Waypoints.csv'
OptInfo['Route']['MaxIceExtent']         = 0.8
OptInfo['Route']['Zero Currents']        = True
OptInfo['Route']['VariableSpeed']        = True
OptInfo['Route']['Time Unit']            = 'days'

OptInfo['Route']['VehicleInfo']                = {}
OptInfo['Route']['VehicleInfo']['Speed']       = 26.5
OptInfo['Route']['VehicleInfo']['Unit']        = 'km/hr'
OptInfo['Route']['VehicleInfo']['Beam']        = 24.0
OptInfo['Route']['VehicleInfo']['HullType']    = 'slender'
OptInfo['Route']['VehicleInfo']['ForceLimit']  = 96634.5

SpeedAdjusted_TT = TravelTime(modis_grid,OptInfo)
SpeedAdjustedPaths = SpeedAdjusted_TT.Paths(verbose=True,multiprocessing=False)

In [ ]:
modis_grid.plot(waypoints=pd.read_csv(OptInfo['Route']['WayPoints']),paths=Paths)

In [ ]:

def MapTravelTime(TT,map,source='Halley'):
    from branca.colormap import linear
    def DijkstraInfoDF(TT,source='Halley'):
        from shapely.geometry import Polygon
        import geopandas as gpd
        import numpy as np
        DijkstraInfo = TT.DijkstraInfo[source]
        Shape   = []; CentroidCx=[];CentroidCy=[];TravelTime=[]
        for idx in DijkstraInfo.index:
            c = TT.Mesh.cellBoxes[idx]
            if isinstance(c, CellBox):
                if DijkstraInfo['traveltime'].loc[idx] == np.inf  and DijkstraInfo['positionLocked'].loc[idx] == False:
                    continue
                bounds = np.array(c.getBounds())
                Shape.append(Polygon(bounds))
                CentroidCx.append(c.cx)
                CentroidCy.append(c.cy)
                if DijkstraInfo['traveltime'].loc[idx] == np.inf:
                    TravelTime.append(0.0)
                else:
                    TravelTime.append(float(DijkstraInfo['traveltime'].loc[idx]))
        Polygons = pd.DataFrame()
        Polygons['geometry'] = Shape
        Polygons['Cx']       = CentroidCx
        Polygons['Cy']       = CentroidCy
        Polygons['TravelTime']    = TravelTime
        Polygons = gpd.GeoDataFrame(Polygons,crs={'init': 'epsg:4326'}, geometry='geometry')
        return Polygons

    DF = DijkstraInfoDF(TT,source=source)


    bathInfo = folium.FeatureGroup(name='TravelTime Mesh',show=True)
    colormap = linear.viridis.scale(min(DF['TravelTime']),max(DF['TravelTime']))
    folium.GeoJson(
        DF,
        style_function=lambda x: {
                'fillColor': colormap(x['properties']['TravelTime']),
                'color': 'gray',
                'weight': 0.1,
                'fillOpacity': 1.0
            }
    ).add_to(bathInfo)
    bathInfo.add_to(map)
    return map


In [ ]:
loc = ['Modis Image Overlay','Route Paths from Halley']
title_html = '''
             <h1 style="color:#003b5c;font-size:16px">
             &ensp;<img src='https://i.ibb.co/JH2zknX/Small-Logo.png' alt="BAS-colour-eps" border="0" style="width:40px;height:40px;"> 
             <img src="https://i.ibb.co/XtZdzDt/BAS-colour-eps.png" alt="BAS-colour-eps" border="0" style="width:179px;height:40px;"> 
             &ensp;|&ensp; {}  &ensp;|&ensp; {}
             </h1>
             </body>
             '''.format(loc[0],loc[1])   


map = Plot.BaseMap(logo=False,logoPos=[5,88])
map.get_root().html.add_child(folium.Element(title_html))

map = Plot.MapMesh(modis_grid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = MapTravelTime(TT,map,source='Halley')
map = Plot.MapPaths(Paths,map,PathPoints=False)
map = Plot.LayerControl(map,collapsed=True)
map

In [ ]:
map.save('FoliumExample.html')